In [ ]:
import easygui as gui
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import io
from google.cloud import vision
import shutil
import pandas as pd

# Configuração do cliente Vision
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/Algoritimos/renomear arquivos em massas/env/client_file.json"
client = vision.ImageAnnotatorClient()
dont_work = []
validate_response = []

# Dicionário de categorias de barcos
barcos = {
    "MEDIOS": ["CILONE", "F.ANDREIS XI", "MERLIM","NEBLINA","NEW FURACAO","NEW TROVAO","PERSIVAL","TEMPESTADE","TORMENTA","TORNADO","TUFAO","VENDAVAL","VENTANIA","VIGO MAXIMUS","ONIX"],
    "PEQUENOS": ["DIQUE COMAND.","ADS I","MERCURIO IV" ,"C. COBRE" ,"PRUDENT" ,"CARLA","CHUVISCO","COBRE","DIALCAR","DIQUE CMTE AFFONSO","FELIZ","FLORA","MERCURIO","MINI BOLA","MINI ORANGE","N ALMEIDA VII","NAVEMAR XIV","NIQUEL","PRUDENTE","RELAMPAGO","TEMPORAL","TITA","ZANGADO","DIQUE COMAN","DIQUE COMAN","TITA I"],
    "AZIMUTAL": ["F. ANDREIS","AGATA","F.ANDREIS","BRILHANTE","CRISTAL","DIAMANTE","ESMERALDA","JADE","OPALA","PEROLA","QUARTZO","TOPAZIO","TURMALINA","TURQUEZA"],
    "OFFSHORE": ["AÇU", "ESMERALDA", "ITAJAI", "NITEROI", "OFFSHORE", "OFFSHORE RIO", "POLARIS", "RIO MINHO", "SALVADOR", "SAGAMORIM", "SEPETIBA", "SUPERPESA", "TUFÃO"]
}
def abrirImagemVision(imagem_path):
    with io.open(imagem_path, 'rb') as imagem_file:
        conteudo = imagem_file.read()
    img = vision.Image(content=conteudo)
    return img.content

def extrairTextoManuscrito_regiao(imagem_path, coordenadas_retangulo):
    img_content = abrirImagemVision(imagem_path)
    pil_image = Image.open(io.BytesIO(img_content))

    # Extrair a região da imagem dentro do retângulo
    x1, y1 = coordenadas_retangulo[0]
    x2, y2 = coordenadas_retangulo[1]
    x1 = max(0, min(x1, pil_image.width))
    x2 = max(0, min(x2, pil_image.width))
    y1 = max(0, min(y1, pil_image.height))
    y2 = max(0, min(y2, pil_image.height))
    cropped_image = pil_image.crop((x1, y1, x2, y2))

    # Desenhar o retângulo na imagem original
    image_with_rectangle = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    cv2.rectangle(image_with_rectangle, coordenadas_retangulo[0], coordenadas_retangulo[1], (255, 0, 0), 2)

    # Converte a imagem recortada para formato binário
    cropped_image_byte_arr = io.BytesIO()
    cropped_image.save(cropped_image_byte_arr, format='PNG')
    cropped_image_content = cropped_image_byte_arr.getvalue()

    # Converte a imagem com o retângulo para formato binário
    image_with_rectangle_byte_arr = io.BytesIO()
    image_with_rectangle_save = Image.fromarray(cv2.cvtColor(image_with_rectangle, cv2.COLOR_BGR2RGB))
    image_with_rectangle_save.save(image_with_rectangle_byte_arr, format='PNG')
    image_with_rectangle_content = image_with_rectangle_byte_arr.getvalue()

    # Executar OCR na região da imagem recortada
    cropped_image_request = vision.Image(content=cropped_image_content)
    response = client.document_text_detection(image=cropped_image_request, image_context={'language_hints': ['pt-t-i0-handwrit']})
    
    # Retornar o texto extraído e a imagem com o retângulo
    texto_manuscrito = response.full_text_annotation.text
    return texto_manuscrito, image_with_rectangle_content

def draw_text(image, text, position):
    # Converte a imagem para o formato BGR (OpenCV usa BGR por padrão)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Defina a fonte e outras propriedades do texto
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 5
    font_color = (255, 0, 0)
    thickness = 6
    # Desenhe o texto na imagem
    cv2.putText(image_bgr, text, position, font, font_scale, font_color, thickness)
    # Converte a imagem de volta para o formato RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    return image_rgb

def determinar_categoria(txt_regiao):
    for linha in txt_regiao.split('\n'):
        for key, valor in barcos.items():
            for item in valor:
                if item.upper() in linha.upper():
                    print(f"Embarcação encontrada ={item} em key {key}" )
                    validate_response.append(item)
                    return key


                elif "REBOCADOR:" in linha:
                    print("linha = ", linha)
                    ship_name = linha[10:].strip()  # Remova espaços em branco extras
                    print("REBOCADOR = ", ship_name)
                    for key, valor in barcos.items():
                        if ship_name in valor:
                            validate_response.append(ship_name)

                            return key
                    else:
                        print("Embarcação não encontrada em nenhuma categoria")

    print("\n\nValidação de resposta = ", validate_response)
    return None

def redimensionar_e_mover_imagens(diretorio_origem, diretorio_destino):
    # Criar diretórios para cada categoria de barco
    for categoria in barcos.keys():
        os.makedirs(os.path.join(diretorio_destino, categoria), exist_ok=True)
    
    for filename in os.listdir(diretorio_origem):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            print("\nArquivo atual =", filename)
            # Caminho completo do arquivo de origem
            caminho_origem = os.path.join(diretorio_origem, filename)
            
            # Pergunte ao usuário o nome e o equipamento
            msg = "Insira o nome e o equipamento para a imagem."
            title = "Informações da Imagem"
            fieldNames = ["Nome", "Equipamento"]
            fieldValues = gui.multenterbox(msg, title, fieldNames)
            
            # Exibir a imagem em uma janela separada
            img_content = abrirImagemVision(caminho_origem)
            pil_image = Image.open(io.BytesIO(img_content))
            pil_image.show()

            # Desenhe o texto da categoria na imagem
            rectangle_coordinates = [(700, 100), (2200, 800)]
            txt_regiao, img_recortada_com_retangulo = extrairTextoManuscrito_regiao(caminho_origem, rectangle_coordinates)
            categoria = determinar_categoria(txt_regiao)

            img_pil = Image.open(io.BytesIO(img_recortada_com_retangulo))
            img_array = np.array(img_pil)  # Converter imagem PIL para numpy array
            img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)  # Convertendo de BGR para RGB
            if categoria is not None:
                print("\n\nCategoria =", categoria)
                image_with_text = draw_text(img_rgb, f"{categoria} - Nome: ", (300, 1200))
                # Adicione o nome e o equipamento na imagem
                if fieldValues is not None:
                    name, equipment = fieldValues
                    image_with_text = draw_text(image_with_text, f"Nome: {name}", (300, 1300))
                    image_with_text = draw_text(image_with_text, f"Equipamento: {equipment}", (300, 1400))
                # Salvar a imagem com texto
                cv2.imwrite(f"{diretorio_destino}/{categoria}/{filename}", image_with_text)
                # Copiar o arquivo original para o diretório de destino
                shutil.copy(caminho_origem, os.path.join(diretorio_destino, categoria, filename))
            else:
                # Se a categoria não for encontrada, desenhe "Categoria não encontrada"
                image_with_text = draw_text(img_rgb, "Categoria nao encontrada", (200, 100))
                dont_work.append(filename)
            # Exibir a imagem recortada com o retângulo
            image_with_text = Image.fromarray(image_with_text)  # Convertendo de volta para PIL Image
            image_with_text.show()
            
# Diretórios de entrada e saída
onedrive_dir = "/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/Algoritimos/renomear arquivos em massas/assets"
output_dir = "/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/Algoritimos/renomear arquivos em massas/output"

# Executar a função
redimensionar_e_mover_imagens(onedrive_dir, output_dir)

print("Arquivos que não funcionaram =", dont_work)
